In [ ]:
import json
import csv
import ast

def json_to_csv(json_file_path, csv_file_path):
    # 讀取 JSON 檔案，指定編碼為 utf-8
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    # 如果 JSON 數據是一個列表，則取第一個元素的鍵作為 CSV 的標題
    if isinstance(data, list) and len(data) > 0:
        keys = data[0].keys()

        # 寫入 CSV 檔案
        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=keys)

            # 寫入標題
            writer.writeheader()

            # 寫入數據
            for row in data:
                # 處理特定欄位
                for key in row:
                    if isinstance(row[key], str):
                        try:
                            # 使用 ast.literal_eval 將字符串轉換為列表
                            list_data = ast.literal_eval(row[key])
                            if isinstance(list_data, list) and len(list_data) > 0:
                                row[key] = list_data[0]
                        except (ValueError, SyntaxError):
                            pass

                writer.writerow(row)

        print(f"JSON 檔案已成功轉換為 CSV 檔案: {csv_file_path}")
    else:
        print("JSON 檔案格式不正確或為空。")

# 使用範例
json_file_path = r'D:\project\stockDataLab\Lab\data\\webJson\\stock_category.json'
csv_file_path = r'D:\project\stockDataLab\Lab\data\\webJson\\stock_category.csv'
json_to_csv(json_file_path, csv_file_path)


In [ ]:
import json
import csv

def csv_to_json(csv_file_path, json_file_path):
    # 讀取 CSV 檔案
    data = []
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            # 移除 'name' 欄位
            print(row)
            row.pop('\ufeffname', None)
            # 將 'id' 欄位轉換為整數
            if 'id' in row:
                row['id'] = int(row['id'])
            data.append(row)

    # 寫入 JSON 檔案
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, separators=(',', ':'))

    print(f"CSV 檔案已成功轉換為 JSON 檔案: {json_file_path}")

# 使用範例
csv_file_path = r'D:\project\stockDataLab\Lab\data\\webJson\\stock_group.csv'
json_file_path = r'D:\project\stockDataLab\Lab\data\\webJson\\stock_category.json'
csv_to_json(csv_file_path, json_file_path)

## 板塊圖設計


In [6]:
#修改股票題材
import pandas as pd

csv_file = r'webJson\stock_subject.csv'
df = pd.read_csv(csv_file)


# # 給定的替換規則
# replacement_rule = "PBC-HDI:欣興、華通、健鼎、聯茂、定穎投控"
# replacement_rule = "AI機器視覺:佳能、圓剛、所羅門、達明"
# replacement_rule = "IP智財:創意、智原、力旺、晶心科、M31、世芯-KY"
# replacement_rule = "板卡顯卡:華擎、映泰、承啟、麗臺、撼訊、青雲"
# replacement_rule = "散熱顯卡:動力-KY、奇鋐、尼得科超眾"
# replacement_rule = "Switch2:茂達、創惟、原相、旺宏、瑞昱、偉詮電"
# replacement_rule = "液冷三雄:奇鋐、雙鴻、建準"
# replacement_rule = "乳膠雙雄:南帝、申豐"
replacement_rule = "CoWoS設備:弘塑、辛耘、萬潤、志聖、均豪、均華"
# 解析替換規則
new_category, codes = replacement_rule.split(':')
codes = set(map(str, codes.split('、')))

# print(new_category, codes)

# # 更新 DataFrame 中的第三欄位
# df.loc[df['id'].astype(int).isin(codes), 'cg'] = new_category
# print(df[df['id'].isin(codes)])

# 更新 DataFrame 中的第三欄位
df.loc[df['name'].isin(codes), 'cg'] = new_category
print(df[df['name'].isin(codes)])

# 將更新後的 DataFrame 寫回到 CSV 文件
df.to_csv(csv_file, index=False)


KeyError: 'name'

In [7]:
# 讀取文字檔案中的股票代碼
with open(r'stock/0.txt', 'r') as file:
    stock_ids = file.read().strip().split(',')

# 過濾掉空字串並將股票代碼轉換為整數
stock_ids = [int(stock_id) for stock_id in stock_ids if stock_id]

# 將股票代碼排序
stock_ids = sorted(stock_ids)

# 將排序後的股票代碼轉換回字串
sorted_stock_ids = ','.join(map(str, stock_ids))

# 輸出排序後的股票代碼
print(len(stock_ids))

# 將排序後的股票代碼寫回文字檔案
with open(r'stock/sorted.txt', 'w') as file:
    file.write(sorted_stock_ids)


685


#### 分切出五個小檔案

In [20]:
# 讀取文字檔案中的股票代碼
with open(r'stock/sorted.txt', 'r') as file:
    stock_ids = file.read().strip().split(',')

# 過濾掉空字串並將股票代碼轉換為整數
stock_ids = [int(stock_id) for stock_id in stock_ids if stock_id]

# 將股票代碼排序
stock_ids = sorted(stock_ids)

# 計算每個檔案應該包含的股票代碼數量
num_files = 5
total_stock_ids = len(stock_ids)
average_stock_ids_per_file = total_stock_ids // num_files
remainder = total_stock_ids % num_files

import random

# 生成每個檔案的股票代碼數量，確保相差大約20筆
file_sizes = []
for i in range(num_files):
    size = average_stock_ids_per_file + (1 if i < remainder else 0)
    size += random.randint(-15, 15)  # 隨機調整大小，確保相差大約20筆
    file_sizes.append(size)

# 調整檔案大小，確保總和等於總股票數量
while sum(file_sizes) != total_stock_ids:
    if sum(file_sizes) < total_stock_ids:
        file_sizes[random.randint(0, num_files - 1)] += 1
    else:
        file_sizes[random.randint(0, num_files - 1)] -= 1

# 將股票代碼分配到五個檔案中
file_index = 1
current_index = 0

for size in file_sizes:
    # 取得當前檔案的股票代碼
    current_stock_ids = stock_ids[current_index:current_index + size]

    # 將股票代碼轉換回字串
    current_stock_ids_str = ','.join(map(str, current_stock_ids))

    # 寫入檔案
    with open(f'stock/Id{file_index}.txt', 'w') as file:
        file.write(current_stock_ids_str)

    # 更新索引
    current_index += size
    file_index += 1